In [1]:
from library_dicom.dicom_processor.tools.pyradiomic import *
import SimpleITK as sitk 
import os 
import numpy as np 
import csv
import json 
from library_dicom.dicom_processor.model.csv_reader.CsvReader import CsvReader

In [2]:
csv_path= '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/PVAB_NIFTI.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_ = []
    for row in reader :
        csv_file_.append(row)
        
del csv_file_[0] #enlever première ligne

In [3]:
for row in csv_file_ : 
    ct = row[4]
    pet = row[5]
    mask = row[6]
    x = ct.replace('/media/oncopole/LACIE SHARE/PVAB', '/media/oncopole/LACIE SHARE/PVAB/NIFTI')
    y = pet.replace('/media/oncopole/LACIE SHARE/PVAB', '/media/oncopole/LACIE SHARE/PVAB/NIFTI')
    z = mask.replace('/media/oncopole/LACIE SHARE/PVAB', '/media/oncopole/LACIE SHARE/PVAB/NIFTI')
    row[4]= x 
    row[5] = y 
    row[6] = z 

In [ ]:
directory = '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/'
filename = 'PVAB_NIFTI_V2.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT_ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "STUDY"])
    for serie in dataset: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6], serie[7]])

In [2]:

csv_path= '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/PVAB_NIFTI.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_ = []
    for row in reader :
        csv_file_.append(row)
        
del csv_file_[0] #enlever première ligne

In [3]:
print("Nombre de mask à traiter : ", len(csv_file_))

Nombre de mask à traiter :  76


#### - Keep only pet 0 

In [4]:
pet0 = []
for row in csv_file_ : 
    if 'PET0' in row : 
        pet0.append(row)
print(len(pet0))

76


In [5]:
#get date 
json_path = '/media/deeplearning/LACIE SHARE/PVAB/pvab_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

#print(len(dataset))
for row in pet0 : 
    patient_id = row[0]
    for liste in dataset : 
        if patient_id in liste : 
            row.append(liste[-1])

#check if every study has its csv 
for row in pet0 : 
    if len(row) != 9 : 
        print(row)


In [9]:
#get_date 
for row in pet0 : 
    #change csv_path 
    csv_file_path = row[-1]
    x = csv_file_path.replace('oncopole', 'deeplearning')
    row[-1] = x 

    #change pet and mask path 
    pet = row[5]
    mask= row[6]
    x = pet.replace('oncopole', 'deeplearning')
    y = mask.replace('oncopole', 'deeplearning')
    row[5] = x 
    row[6] = y 

    #get date 

    reader = CsvReader(row[-1])
    date = reader.get_date()
    row.append(date)


In [10]:
for row in pet0 : 
    if len(row) != 10 : 
        print(row)

In [11]:
pet0[0]

['13014101023001',
 '13014101023001',
 '1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715',
 'PET0',
 '/media/oncopole/LACIE SHARE/PVAB/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_CT.nii',
 '/media/deeplearning/LACIE SHARE/PVAB/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_PT.nii',
 '/media/deeplearning/LACIE SHARE/PVAB/13014101023001/PET0/1.2.276.0.7230010.3.1.2.380371712.1.1605271362.777715_nifti_mask.nii',
 'pvab',
 '/media/deeplearning/LACIE SHARE/PVAB/PVAB_csv/1004/13014101023001_PET0_41.csv',
 ' May 15_2017']

In [12]:
for nifti in pet0 : 
    try : 
        print(pet0.index(nifti))
        mask_path = nifti[-4]
        pet_path = nifti[-5]
        liste_center = get_center_of_mass(mask_path, thresh = True, pet_path = pet_path)
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(nifti)
        print(err)

ase.cxx:97:
sitk::ERROR: The file "/media/deeplearning/LACIE SHARE/PVAB/13014101213003/PET0/1.2.276.0.7230010.3.1.2.346817536.1.1604927295.742870_nifti_PT.nii" does not exist.
53
['13014101343001', '13014101343001', '1.2.276.0.7230010.3.1.2.447480576.1.1615450568.262291', 'PET0', '/media/oncopole/LACIE SHARE/PVAB/13014101343001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.1615450568.262291_nifti_CT.nii', '/media/deeplearning/LACIE SHARE/PVAB/13014101343001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.1615450568.262291_nifti_PT.nii', '/media/deeplearning/LACIE SHARE/PVAB/13014101343001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.1615450568.262291_nifti_mask.nii', 'pvab', '/media/deeplearning/LACIE SHARE/PVAB/PVAB_csv/1024/13014101343001_PET0_41.csv', ' Sep 22_2016']
Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "/media/deeplearning/LACIE SHARE/PVAB/13014101343001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.161545056

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE'
filename = 'AHL2011_dMax_v5.csv'


with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in pet0: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])
